# **Handwritten Digit Recognizer (MNIST Classifier)**
* Loading the MNIST dataset
* Defining  the Data loaders

In [0]:
import torch
from torchvision import datasets, transforms
from torch import nn, optim
import torch.nn.functional as F

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, ), (0.5,))])
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download = True, train = True, transform = transform)
testset = datasets.MNIST('~/.pytorch/MNIST_data/', download = True, train = False, transform = transform)

trainLoader = torch.utils.data.DataLoader(trainset, batch_size = 20, shuffle = True, num_workers = 0)
testLoader = torch.utils.data.DataLoader(testset, batch_size = 20, shuffle = True, num_workers = 0)

Visualizing the Image data

In [0]:
import numpy as np
import matplotlib.pyplot as plt

data = iter(trainLoader)
images, labels = data.next()
images = images.numpy()

fig = plt.figure(figsize = (20, 5))
for img in range(20):
  ax = fig.add_subplot(2, 10, img + 1)
  ax.imshow(np.squeeze(images[img]), cmap = 'gray')
  ax.set_title(str(labels[img].item()))
  ax.axis('off')

## Visualizing PIXEL data

In [0]:
img = np.squeeze(images[3])
fig = plt.figure(figsize = (15, 15))
ax = fig.add_subplot(111)
ax.imshow(img, cmap = 'gray')
width, height = img.shape
thresh = img.max()/2.5
for i in range(width):
  for j in range(height):
    val = round(img[i][j], 2) if img[i][j] > 0 else 0
    ax.annotate(str(val), xy = (j, i),
               horizontalalignment = 'center',
               verticalalignment = 'center',
               color = 'white' if img[i][j] < thresh else 'black')

##**`Defining the Network architecture`**

In [0]:
class Classifier(nn.Module):
  def __init__(self):
    super(Classifier, self).__init__()
    self.fc1 = nn.Linear(784, 512)
    self.fc2 = nn.Linear(512, 512)
    self.fc3 = nn.Linear(512, 10)
    self.dropout = nn.Dropout(p = 0.2)
    
  def forward(self, x):
    x = x.view(images.shape[0], -1)
    
    x = self.dropout(F.relu(self.fc1(x)))
    x = self.dropout(F.relu(self.fc2(x)))
    x = F.log_softmax(self.fc3(x), dim = 1)
    return x

## **`Feed forward and Backpropagation`**

In [0]:
model = Classifier()
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.01)

epochs = 50
train_loss, test_loss = [], []
for e in range(epochs):
  model.train()
  trainloss = 0
  for images, labels in trainLoader:
    optimizer.zero_grad()
    logps = model(images)
    loss = criterion(logps, labels)
    loss.backward()
    optimizer.step()
    trainloss += loss.item()
  else:
    testloss, accuracy = 0, 0
    with torch.no_grad():
      model.eval()
      for images, labels in testLoader:
        ps = torch.exp(model(images))
        top_ps, top_class = ps.topk(1, dim = 1)
        equals = top_class == labels.view(top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor))
        testloss += criterion(model(images), labels).item()
      model.train()
      train_loss.append(trainloss/len(trainLoader))
      test_loss.append(testloss/len(testLoader))
      print("Epoch {}/{}".format(e + 1, epochs),
           "Train loss {:.3f}".format(trainloss/len(trainLoader)),
           "Validation loss {:.3f}".format(testloss/len(testLoader)),
           "Accuracy {:.3f}".format((accuracy/len(testLoader)) * 100))

# *Inference*

In [0]:
dataiter = iter(testLoader)
images, labels = dataiter.next()
ps = model(images)
top_ps, top_class = ps.topk(1, dim = 1)
images = images.numpy()
fig = plt.figure(figsize = (25, 4))
for img in range(20):
  ax = fig.add_subplot(2, 10, img + 1)
  ax.imshow(np.squeeze(images[img]), cmap = 'gray')
  ax.axis('off')
  ax.set_title("{0} ({1})".format(str(top_class[img].item()), str(labels[img].item())),
              color = "green" if top_class[img] == labels[img] else "red", fontsize = 18)